In [ ]:
# import sys
# !{sys.executable} -m pip install fpdf -q
# !{sys.executable} -m pip install pandas -q
# !{sys.executable} -m pip install plotly-express -q
# !{sys.executable} -m pip install kaleido -q

In [ ]:
from datetime import date
from pathlib import Path
import sqlite3

import pandas as pd  
import plotly.express as px  
from fpdf import FPDF 

## Define charts and paths


In [ ]:
plotly_template = "simple_white"

In [ ]:
current_dir = Path(__file__).parent if "__file__" in locals() else Path.cwd()
database_path = current_dir / "sales.db"
output_dir = current_dir / "output"

# Create the output directory and its parent directory if they do not exist
output_dir.mkdir(parents=True, exist_ok=True)

## Total sales per month

In [ ]:
conn = sqlite3.connect(database_path)

query = '''
SELECT sale_date, SUM(total_price) as total_sales
FROM sales
GROUP BY sale_date
ORDER BY sale_date ASC
'''
df = pd.read_sql_query(query, conn)

# Print the DataFrame
print(df)


In [ ]:

# Check the data types
df.info()


In [ ]:

# Convert sale_date to datetime
df['sale_date'] = pd.to_datetime(df['sale_date'])
df.info()


In [ ]:

# Set the sale_date column as the index
df = df.set_index('sale_date')
df.head(3)

In [ ]:
# Resample the data to a monthly frequency and compute the sum
df_monthly = df.resample('M').sum()
df_monthly

In [ ]:
# Map the month number to short month name
df_monthly['month_name'] = df_monthly.index.strftime('%b')
df_monthly

In [ ]:
# Create the Plotly figure with text parameter
fig = px.bar(df_monthly,
             x='month_name',
             y='total_sales',
             template=plotly_template,
             text='total_sales')

# Set the layout
fig.update_layout(
    title='Total Sales by Month',
    xaxis_title='Month',
    yaxis_title='Total Sales ($)',
    yaxis_tickprefix='$',
)

# Show the plot
fig.show()

# Save the chart as a PNG image
fig.write_image(output_dir / 'monthly_sales.png',
                width=1200,
                height=400,
                scale=4)

## Total Sales by Product

In [ ]:
# Execute the query and load results into a Pandas DataFrame
query = '''
SELECT p.product_name, SUM(s.total_price) as total_sales
FROM sales s
JOIN products p ON s.product_id = p.product_id
GROUP BY p.product_name
'''
df = pd.read_sql_query(query, conn)

In [ ]:
df

In [ ]:
# Create the Plotly figure with text parameter
fig = px.bar(df,
             x='product_name',
             y='total_sales',
             template=plotly_template,
             text='total_sales')

# Set the layout
fig.update_layout(
    title='Total Sales by Product',
    xaxis_title='Product',
    yaxis_title='Total Sales ($)',
    yaxis_tickprefix='$',
)

# Show the plot
fig.show()

# Save the chart as a PNG image
fig.write_image(output_dir / 'product_sales.png',
                width=1200,
                height=400,
                scale=4)

## Top Customer by Sales

In [ ]:
# Execute the query and load results into a Pandas DataFrame
query = '''
SELECT c.first_name || ' ' || c.last_name as customer_name, SUM(s.total_price) as total_sales
FROM sales s
JOIN customers c ON s.customer_id = c.customer_id
GROUP BY customer_name
ORDER BY total_sales DESC
LIMIT 10
'''
df = pd.read_sql_query(query, conn)

In [ ]:
df

In [ ]:
# Create the Plotly figure
fig = px.bar(df,
             x='customer_name',
             y='total_sales',
             template=plotly_template,
             text='total_sales')

# Set the layout
fig.update_layout(
    title='Top Customers by Sales',
    xaxis_title='Customer',
    yaxis_title='Total Sales ($)',
    yaxis_tickprefix='$',
)

# Show the plot
fig.show()

# Save the chart as a PNG image
fig.write_image(output_dir / 'customer_sales.png',
                width=1200,
                height=400,
                scale=4)

## Create PDF Report

In [ ]:
# Define the font color as RGB values (dark gray)
font_color = (64, 64, 64)

# Find all PNG files in the output folder
chart_filenames = [str(chart_path) for chart_path in output_dir.glob("*.png")]

# Create a PDF document and set the page size
pdf = FPDF()
pdf.add_page()
pdf.set_font('Arial', 'B', 24)

# Add the overall page title
title = f"Sales Report as of {date.today().strftime('%m/%d/%Y')}"
pdf.set_text_color(*font_color)
pdf.cell(0, 20, title, align='C', ln=1)

# Add each chart to the PDF document
for chart_filename in chart_filenames:
    pdf.ln(10)  # Add padding at the top of the next chart
    pdf.image(chart_filename, x=None, y=None, w=pdf.w - 20, h=0)

# Save the PDF document to a file on disk
pdf.output(output_dir / "sales_report.pdf", "F")

### Customer Segment using SQL

In [ ]:
# Execute the query and load results into a Pandas DataFrame
query = '''
SELECT 
  customers.customer_id, 
  customers.first_name || ' ' || customers.last_name as customer_name, 
  SUM(sales.total_price) as total_sales,
  CASE 
    WHEN SUM(sales.total_price) > 30000 THEN 'High Value'
    WHEN SUM(sales.total_price) > 26000 THEN 'Medium Value'
    ELSE 'Low Value'
  END as customer_segment
FROM sales
INNER JOIN customers ON sales.customer_id = customers.customer_id
GROUP BY customers.customer_id
ORDER BY total_sales DESC
'''

df = pd.read_sql_query(query, conn)
df

### Customer Segment using SQL + Pandas

In [ ]:
# Execute the query and load results into a Pandas DataFrame
query = '''
SELECT 
  customers.customer_id, 
  customers.first_name || ' ' || customers.last_name as customer_name, 
  SUM(sales.total_price) as total_sales
FROM sales
INNER JOIN customers ON sales.customer_id = customers.customer_id
GROUP BY customers.customer_id
'''

df = pd.read_sql_query(query, conn)
df

In [ ]:
# Group the data by customer segment
bins = [0, 26000, 30000, float('inf')]
labels = ['Low Value', 'Medium Value', 'High Value']
df['customer_segment'] = pd.cut(df['total_sales'], bins=bins, labels=labels)

# Order the data by total sales
df = df.sort_values(by='total_sales', ascending=False)
df

In [ ]:
# Close the connection
conn.close()